In [1]:
## 导入需要的库
import numpy as np
import pandas as pd
import math
import os
import random
from matplotlib.pyplot import *
import matplotlib.pyplot as plt

In [3]:
## 读取数据
df = pd.read_excel("2022年中国证指股票（沪深100）数据集/预处理后的数据.xlsx")
df.head()

,股票代码,1月4日,1月5日,1月6日,1月7日,1月10日,1月11日,1月12日,1月13日,1月14日,...,12月12日,12月13日,12月14日,12月15日,12月16日,12月19日,12月20日,12月21日,12月22日,12月23日
0,000651.SZA,6457.4060,6709.9970,6634.3916,6897.293,6919.6310,6866.3633,6790.7580,6718.5890,6572.5320,...,6269.8500,6286.8060,6292.4580,6218.9830,6266.0825,6183.1875,6085.2210,6109.7130,6205.7954,6175.6520
1,600309.SHA,3845.7060,3864.9250,3880.3000,3953.332,3870.6907,3774.5962,3812.6497,3731.1614,3693.8767,...,3603.1821,3616.6729,3642.4636,3621.0374,3626.5923,3600.0078,3555.5684,3565.0910,3633.3376,3523.8257
2,600276.SHA,3233.9270,3267.9412,3215.9570,3265.374,3280.7769,3220.4495,3182.5845,3047.1690,2993.9014,...,2504.9739,2472.7598,2476.6252,2445.6997,2448.2769,2396.0898,2352.2786,2328.4402,2360.0100,2367.7415
3,600436.SHA,2833.2197,2719.7732,2634.6390,2584.240,2588.6310,2581.0286,2587.4514,2555.9930,2530.4330,...,1876.9463,1850.9615,1878.1304,1919.5742,1923.7186,1878.8540,1840.5677,1901.0889,1893.9843,1904.7728
4,300014.SZA,1985.6554,1913.7346,1879.9104,1876.172,1846.0863,1886.4972,1940.4379,1914.4467,1997.4048,...,1609.1224,1574.5331,1575.2462,1643.8903,1626.0607,1656.3710,1593.2542,1589.6882,1545.8274,1537.4474


In [4]:
#根据日收盘价计算日收益率，再计算日对数收益率
df1 = np.zeros((len(df),len(df.T)-2))  #日收益率
df2 = np.zeros((len(df),len(df.T)-2))  #日对数收益率
for i in range(len(df)):
    for j in range(len(df.T)-2):
        df1[i][j] = (df.loc[i][j+2]-df.loc[i][j+1])/(df.loc[i][j+1])
        df2[i][j] = math.log(1+df1[i][j])
df2

array([[ 0.03837081, -0.01133154,  0.03886205, ...,  0.00401676,
         0.0156038 , -0.00486913],
       [ 0.00498508,  0.00397019,  0.0186463 , ...,  0.00267464,
         0.0189621 , -0.03060443],
       [ 0.010463  , -0.0160352 ,  0.01524932, ..., -0.01018587,
         0.01346725,  0.00327069],
       ...,
       [-0.01064469, -0.02285124,  0.02641208, ...,  0.01398611,
         0.01834919,  0.01204834],
       [-0.01462464, -0.00806819,  0.01296691, ..., -0.00647024,
        -0.00760183,  0.00543577],
       [ 0.00784315, -0.01178798,  0.01763022, ..., -0.00583095,
        -0.00586509, -0.0029455 ]])

In [5]:
##根据算法对数据进行聚类
#归一化数据
n, m = len(df2), len(df2.T)
data1 = np.zeros((n,m))
for i in range(n):
    a = min(df2[i])
    b = max(df2[i])
    
    for j in range(m):
        data1[i][j]=(df2[i][j]-a)/(b-a)
data2 = pd.DataFrame(data1)
data2.to_csv(r"中国预处理后的数据归一化数据.csv")

In [7]:
def GYjuli(i,j):
    theta = 0.5
    m = len(df2.T)
    n = len(df2)
    a, b, a1, b1 = [], [], [], []
    X1 = np.zeros((1,m))
    X2 = np.zeros((1,m))
    A, B, A1, B1, D = 0, 0, 0, 0, 0
    # 取最大最小值
    for l in range(n):
        a.append(min(abs(data2.loc[i]-data2.loc[l])))
        b.append(max(abs(data2.loc[i]-data2.loc[l])))
        a1.append(min(abs(data2.loc[j]-data2.loc[l])))
        b1.append(max(abs(data2.loc[j]-data2.loc[l])))
    
    A, B = min(a), max(b)
    A1, B1 = min(a1), max(b1)
    c = abs(data2.loc[i]-data2.loc[j])
    
    for k in range(m):
        X1[0][k] = (A+theta*B)/(c[k]+theta*B) # 计算不同属性下的灰色关联系数
        X2[0][k] = (A1+theta*B1)/(c[k]+theta*B1)
    # print(X1)
    # print(X2)
    # D = np.sqrt(np.sum(np.square(X1 - X2))) #欧式距离
    D = np.sum(abs(X1 - X2)) #汉明距离
    return D

In [8]:
D = np.zeros((n,n))
for i in range(n):
    for j in range(n):
        if i<j: 
            D[i][j] = GYjuli(i,j)
        if i>j:
            D[i][j] = D[j][i]
D1 = pd.DataFrame(D)
D1.to_csv(r"中国股票间的距离.csv")

In [9]:
def xiaoyong(x,singam,theta):
    lamda = np.zeros((len(x),6));
    lamda1 = np.ones((len(x),6));
    lamda2 = np.zeros((len(x),6));
    juhe = np.zeros((1,6));
    
    for i in range(0,len(x)): # 计算不同属性下的函数
        # 计算相对损失函数
        lamda[i,1] = singam * x[i];
        lamda[i,2] = x[i];
        lamda[i,3] = 1-x[i];
        lamda[i,4] = singam * (1-x[i]);
    
        # 计算基于后悔理论的效用函数
        lamda1[i,1] = 1 - (1 - np.exp(theta * lamda[i,1]))/theta
        lamda1[i,2] = 1 - (1 - np.exp(theta * lamda[i,2]))/theta
        lamda1[i,3] = 1 - (1 - np.exp(theta * lamda[i,3]))/theta
        lamda1[i,4] = 1 - (1 - np.exp(theta * lamda[i,4]))/theta
        
        # 计算相对效用函数
        lamda2[i,0] = 1 - lamda1[i,2] 
        lamda2[i,1] = lamda1[i,1] - lamda1[i,2] 
        lamda2[i,4] = lamda1[i,4] - lamda1[i,3] 
        lamda2[i,5] = 1 - lamda1[i,3]
        
    # 计算聚合相对效用函数(均值聚合)
    juhe = np.mean(lamda2,axis=0)
    
    return juhe

In [10]:
# 计算不同对象的聚合相对效用函数矩阵
singam = 0.15 
theta = 0.5
lamda = np.zeros((n,6));
for i in range(0,n):
    x = data2.iloc[i]
    lamda[i] = xiaoyong(x ,singam, theta)

#根据聚合相对效用函数计算不同对象的边界阈值
yuzhi = np.zeros((n,2))
for i in range(0,n):
    for j in range(0,len(lamda.T)):
        yuzhi[i][0] = (lamda[i][4])/(lamda[i][4]+lamda[i][0]-lamda[i][1])
        yuzhi[i][1] = (lamda[i][5]-lamda[i][4])/(lamda[i][1]+lamda[i][5]-lamda[i][4])
yuzhi = pd.DataFrame(yuzhi)
yuzhi.to_csv(r"中国股票的边界阈值.csv")

In [12]:
##根据归一化数据计算每个对象到正理想解和负理想解的距离
d1 = np.zeros((n,1)) 
d2 = np.zeros((n,1)) 
lishu = np.zeros((n,1))

for i in range(0,n):
    a = 0
    b = 0
    for j in range(0,m):
        a = a + (1 - data2.T[i][j])*(1 - data2.T[i][j])
        b = b + data2.T[i][j] * data2.T[i][j]
    d1[i] = np.sqrt(a) #每个对象到正理想解的距离
    d2[i] = np.sqrt(b) #每个对象到负理想解的距离
    lishu[i] = d1[i]/(d1[i]+d2[i])
lishuu = pd.DataFrame(lishu)
lishuu.to_csv(r"中国股票的隶属函数.csv")  

In [13]:
def qitayuzhi1(yz):
    #根据阈值设置下一个阈值的范围
    if yz[1] < 0.5:
        yuzhi2 = np.arange(yz[1],0.5,0.05) 
        V = []
        for k in range(0,len(yuzhi2)):
            yuzhi1 = 1-yuzhi2[k]
            x = 0 
            v1 = v2 = []
            v3 = 0
            
            #计算升高区域、降低区域、阴影区域的面积
            for i in range(0,n):
                if lishu[i] >= yuzhi1:
                    v1.append(1-lishu[i])
                elif lishu[i] <= yuzhi2[k]:
                    v2.append(lishu[i])
                else:
                    v3 = v3 + 1
            V1 = np.sum(v1)
            V2 = np.sum(v2)
            V3 = v3
    
            V.append(abs(V1+V2-V3))

    else:
        yuzhi2 = np.arange(yz[1],yz[0],0.05)
        x = 1
        V = []
        l = []
        
        for k in range(0,len(yuzhi2)):   
            yuzhi1 = np.arange(yuzhi2[k]+0.05,yz[0]+0.05,0.05)
            l.append(len(yuzhi1))
            for j in range(0,len(yuzhi1)):
                v1 = v2 = []
                v3 = 0
                
                #计算升高区域、降低区域、阴影区域的面积
                for i in range(0,n):
                    if lishu[i] >= yuzhi1[j]:
                        v1.append(1-lishu[i])
                    elif lishu[i] <= yuzhi2[k]:
                        v2.append(lishu[i])
                    else:
                        v3 = v3 + 1
                V1 = np.sum(v1)
                V2 = np.sum(v2)
                V3 = v3
    
                V.append(abs(V1+V2-V3))
        
    #根据最优化目标函数得到阈值
    if x == 0: 
        yz2 = yuzhi2[V.index(min(V))]
        yz1 = 1-yz2
    else:
        xx = V.index(min(V))
        for i in range(0,len(l)):
            if xx-l[i]>=0:
                xx = xx-l[i]
            else: 
                yz2 = yuzhi2[i]
                yuzhi1 = np.arange(yuzhi2[i]+0.05,yz[0]+0.05,0.05)
                yz1 = yuzhi1[xx]
    return yz1,yz2

In [14]:
df1 = pd.read_csv("中国股票的边界阈值.csv",index_col = 0)
df1.head()

,0,1
0,0.916182,0.206357
1,0.874990,0.141183
2,0.856703,0.124561
3,0.807988,0.090854
4,0.884358,0.151402


In [15]:
dis = pd.read_csv("中国股票间的距离.csv")
dis = dis.drop(['Unnamed: 0'],axis=1)
dis.head()

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,0.000000,4.971027,14.057456,7.462341,0.588182,6.164761,0.000000,6.097535,4.463997,0.000000,...,10.208441,1.093252,1.765715,9.805818,5.878180,0.004858,0.023527,3.367878,4.077776,1.213120
1,4.971027,0.000000,7.440478,0.513106,4.800199,0.290836,5.209882,0.523689,0.376891,5.293513,...,7.060337,4.185909,3.339332,3.794739,0.678918,5.156629,5.189990,1.606904,1.617720,3.165315
2,14.057456,7.440478,0.000000,6.959487,12.962672,6.763958,11.487654,6.032547,8.895112,12.797131,...,0.719044,11.971103,9.117730,3.295882,6.395819,11.534161,12.296246,8.452531,9.989507,11.909060
3,7.462341,0.513106,6.959487,0.000000,5.790503,0.166086,3.842577,0.071381,0.997817,6.247714,...,8.605359,4.919445,3.765127,3.451030,0.260404,5.633108,5.697073,2.177340,2.039491,4.995194
4,0.588182,4.800199,12.962672,5.790503,0.000000,4.683343,0.634302,5.333174,4.243928,0.507078,...,11.766809,0.406504,1.140603,8.854125,5.230721,0.539275,0.535022,2.863814,3.224866,1.228817


In [16]:
y = 0
for l in range(0,n):
    sc = [];uc = []; dc =[]; 
    # 根据边界阈值进行三支决策
    for i in range(n):
        if dis.loc[l][i]>=df1.loc[l][0]:
            dc.append(i)
        elif dis.loc[l][i]<df1.loc[l][0] and dis.loc[l][i]>df1.loc[l][1]:
            uc.append(i)
        else:
            sc.append(i) 
        
    # 根据其他阈值继续进行三支决策
    yz = qitayuzhi1(df1.loc[l])
    #print(yz)
    while y == 0:
        yuzhi = yz
        for i in range(n):
            if i in uc:
                if dis.loc[l][i] <= yuzhi[1]:
                    sc.append(i)
                    uc.remove(i)
                if dis.loc[l][i] >= yuzhi[0]: 
                    dc.append(i)
                    uc.remove(i)
        if yuzhi[0]- 0.5 > 0.1 or len(uc) >= 1/4*n:
            yz = qitayuzhi1(yuzhi)
            # print(yz)
        else:
            y = 1
    print(l,'............')        
    print(sc)
    print(uc)
    print(dc)

0 ............
[0, 6, 9, 13, 19, 25, 26]
[]
[1, 2, 3, 5, 7, 8, 10, 11, 12, 14, 15, 16, 17, 18, 20, 21, 22, 23, 24, 27, 28, 29, 4]
1 ............
[1]
[3, 5, 7, 8, 11, 16, 17, 24]
[0, 2, 4, 6, 9, 10, 12, 13, 14, 15, 18, 19, 20, 21, 22, 23, 25, 26, 27, 28, 29]
2 ............
[2]
[20]
[0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21, 22, 23, 24, 25, 26, 27, 28, 29]
3 ............
[3, 7]
[1, 5, 11, 17, 24]
[0, 2, 4, 6, 8, 9, 10, 12, 13, 14, 15, 16, 18, 19, 20, 21, 22, 23, 25, 26, 27, 28, 29]
4 ............
[4]
[0, 6, 9, 13, 19, 21, 25, 26]
[1, 2, 3, 5, 7, 8, 10, 11, 12, 14, 15, 16, 17, 18, 20, 22, 23, 24, 27, 28, 29]
5 ............
[5]
[1, 3, 7, 8, 11, 17, 24]
[0, 2, 4, 6, 9, 10, 12, 13, 14, 15, 16, 18, 19, 20, 21, 22, 23, 25, 26, 27, 28, 29]
6 ............
[0, 6, 9, 13, 19, 25, 26]
[4]
[1, 2, 3, 5, 7, 8, 10, 11, 12, 14, 15, 16, 17, 18, 20, 21, 22, 23, 24, 27, 28, 29]
7 ............
[3, 7, 11, 17]
[1, 5, 24]
[0, 2, 4, 6, 8, 9, 10, 12, 13, 14, 15, 16, 18, 19, 20, 21, 22